In [2]:
import torch
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GATConv
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load your dataframe
data = pd.read_csv("data.csv")  # Replace with your actual file path or data
node_features = data.columns.values.tolist()

# Normalize features
scaler = StandardScaler()
data[node_features] = scaler.fit_transform(data[node_features])

# Create graphs for each play
def create_graphs(dataframe):
    graphs = []
    for play_id, play_data in dataframe.groupby('playId'):
        # Node features
        x = torch.tensor(play_data[node_features].values, dtype=torch.float)
        # Create fully connected edges (or define your own edge logic)
        num_nodes = x.size(0)
        edge_index = torch.tensor(
            [[i, j] for i in range(num_nodes) for j in range(num_nodes) if i != j], dtype=torch.long
        ).t().contiguous()
        # Graph label (assuming all nodes in the play share the same result)
        y = torch.tensor([play_data['result'].iloc[0]], dtype=torch.long)
        graphs.append(Data(x=x, edge_index=edge_index, y=y))
    return graphs

graphs = create_graphs(data)

# Split data into train, validation, and test sets
train_graphs, test_graphs = train_test_split(graphs, test_size=0.2, random_state=42)
train_graphs, val_graphs = train_test_split(train_graphs, test_size=0.2, random_state=42)

train_loader = DataLoader(train_graphs, batch_size=16, shuffle=True)
val_loader = DataLoader(val_graphs, batch_size=16, shuffle=False)
test_loader = DataLoader(test_graphs, batch_size=16, shuffle=False)

# Define the GAT model
class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_heads):
        super(GAT, self).__init__()
        self.gat1 = GATConv(in_channels, hidden_channels, heads=num_heads, concat=True)
        self.gat2 = GATConv(hidden_channels * num_heads, out_channels, heads=1, concat=False)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.elu(self.gat1(x, edge_index))
        x = self.gat2(x, edge_index)
        return x

# Instantiate the model
in_channels = len(node_features)
hidden_channels = 8
out_channels = len(data['result'].unique())
num_heads = 4

model = GAT(in_channels, hidden_channels, out_channels, num_heads)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

# Train the model
def train(model, loader, optimizer, criterion, device='cpu'):
    model.train()
    total_loss = 0
    for data in loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# Evaluate the model
def evaluate(model, loader, criterion, device='cpu'):
    model.eval()
    total_loss = 0
    correct = 0
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            out = model(data)
            loss = criterion(out, data.y)
            total_loss += loss.item()
            pred = out.argmax(dim=1)
            correct += (pred == data.y).sum().item()
    accuracy = correct / len(loader.dataset)
    return total_loss / len(loader), accuracy

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

num_epochs = 100
for epoch in range(1, num_epochs + 1):
    train_loss = train(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = evaluate(model, val_loader, criterion, device)
    print(f"Epoch {epoch:03d}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

# Test the model
test_loss, test_acc = evaluate(model, test_loader, criterion, device)
print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")


C:\Users\ivslab\AppData\Local\Temp\ipykernel_22712\583019361.py:29: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  y = torch.tensor([play_data['result'].iloc[0]], dtype=torch.long)
C:\Users\ivslab\AppData\Local\Temp\ipykernel_22712\583019361.py:29: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  y = torch.tensor([play_data['result'].iloc[0]], dtype=torch.long)
C:\Users\ivslab\AppData\Local\Temp\ipykernel_22712\583019361.py:29: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  y = torch.tensor([play_data['result'].iloc[0]], dtype=torch.long)
C:\Users\ivslab\AppData\Local\Temp\ipykernel_2271

ValueError: Expected input batch_size (352) to match target batch_size (16).